In [1]:
import numpy as np
from scipy.linalg import svd, qr, eig, eigh, pinv
from scipy.linalg import inv as inv
from scipy.sparse.linalg import svds
from POD import import_data
import os

import DMD
import importlib
importlib.reload(DMD)


<module 'DMD' from '/home/julielimonet/Documents/Stage_M2/code/DMD.py'>

In [2]:
current_directory = os.getcwd()
time, x, z, u, w, T, umean = import_data(f'/home/julielimonet/Documents/Stage_M2/')
wmean = np.mean(w, axis=0)
w = w - wmean


In [3]:
np.shape(u)
h, l = np.shape(x)
m = len(time)
# reshape
U = np.reshape(w, (m, h * l))


print(m)

6249


# Jean-Christophe Loiseau' version

In [96]:
from DMD import dmd_analysis, evd_dmd


uf = np.zeros((u.shape[0],u.shape[1]*u.shape[2]))
for i in range(u.shape[0]):
  uf[i] = w[i].flatten()
print(u.shape,uf.shape,u.shape[1]+u.shape[2])

# --> apply DMD algorithm
rank = 10
sigma, P, Q = dmd_analysis(np.swapaxes(uf,0,1), y=None, rank=rank, rtol=1e-6)
print(P.shape,sigma.shape)
Phi, Psi, mu = evd_dmd(P, Q)
print(Phi.shape,mu.shape)

(6249, 81, 51) (6249, 4131) 132
(4131, 10) (10,)
(4131, 10) (10,)


In [97]:
print(rank)

10


In [98]:
# --> reconstruct dominant dynamical modes
eigenvalues = np.zeros((rank,))
modes = []
[modes.append([]) for i in range(rank)]
for i in range(rank):
  modes[i] = np.real(Phi[:,i])
  modes[i] = modes[i].reshape((u.shape[1],u.shape[2])) 
  print(modes[i].shape)

(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)


In [101]:
print(Phi[0])

[ 3.41624430e-14+0.00000000e+00j  3.67572402e-15+2.32952045e-14j
  3.67572402e-15-2.32952045e-14j -1.96600333e-15+0.00000000e+00j
 -3.94281020e-14+6.65922090e-15j -3.94281020e-14-6.65922090e-15j
  1.93293153e-14+0.00000000e+00j -2.04197255e-14+0.00000000e+00j
  7.07882258e-14-6.49499030e-15j  7.07882258e-14+6.49499030e-15j]


In [19]:
fig, ax = plt.subplots(figsize=(10,1.5*rank),nrows=rank+1)
cf0 = ax[0].contourf(x, z, umean, levels=20, cmap=cm.nipy_spectral)
plt.colorbar(cf0)
for j in range(rank):
  cf0 = ax[j+1].contourf(x, z, modes[j], levels=20, cmap=cm.nipy_spectral)
  plt.colorbar(cf0)
[ax[j].set_aspect('equal', 'box') for j in range(rank+1)]
# [ax[j+1].set_title(f'Mode {j}, KE = {np.round(KE_mode[j]*100, 3)}%') for j in range(num_modes)]
plt.tight_layout()
plt.show()

# Naive version

In [56]:
print(m)
print(h)
print(l)
print(np.shape(u))
print(np.shape(U))

6249
81
51
(6249, 81, 51)
(6249, 4131)


In [4]:
print(np.shape(U), np.shape(np.swapaxes(U,0,1)))
V1 = np.swapaxes(U,0,1)
V1, V2 = V1[:, :-1], V1[:, 1:]

# First svd
U, Sigma, W = svd(V1, full_matrices=False)
# U, Sigma, W = svds(V1, k = 10)
W = W.T.conj()

(6249, 4131) (4131, 6249)


In [5]:
print(np.shape(V1))
print(np.shape(U), np.shape(Sigma), np.shape(W))

(4131, 6248)
(4131, 4131) (4131,) (6248, 4131)


In [5]:
Sigma_inv = pinv(np.diag(Sigma))

In [ ]:
print(np.shape(U.T), np.shape(V2), np.shape(W), np.shape(Sigma_inv))

(4131, 4131) (4131, 6248) (6248, 4131) (4131, 4131)


In [6]:
S = U.T.conj() @ V2 @ W @ Sigma_inv

In [9]:
print(np.shape(S))

(4131, 4131)


In [7]:
eigenvalues, eigenvectors = eig(S)

In [18]:
print(np.shape(eigenvectors), np.shape(U))

(4131, 4131) (4131, 4131)


In [ ]:
Phi = np.zeros(np.shape(eigenvectors))
for i in range(np.shape(eigenvectors)[1]):
    Phi[i] = U @ eigenvectors[:,i]

In [20]:
print( np.shape(Phi))

(4131, 4131)


In [8]:
indices_of_max_values = eigenvalues.argsort()[-10:][::-1]
rank = 10

# TKE = np.sum(np.real(eigenvalues))
# KE_mode = []
modes = []
[modes.append([]) for i in range(rank)]
for i,j in enumerate(indices_of_max_values):
  # KE_mode.append(np.real(eigenvalues)[j]/TKE)
  modes[i] = np.real(U @ eigenvectors[:,j])
  modes[i] = modes[i].reshape((h,l)) 
  print(modes[i].shape)

(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)


In [33]:
print(np.shape(wmean))

(6249, 81, 51)


In [9]:
import matplotlib.pyplot as plt
from matplotlib import cm

fig, ax = plt.subplots(figsize=(10,1.5*rank),nrows=rank+1)
cf0 = ax[0].contourf(x, z, wmean, levels=20, cmap=cm.nipy_spectral)
plt.colorbar(cf0)
for j in range(rank):
  cf0 = ax[j+1].contourf(x, z, modes[j], levels=20, cmap=cm.nipy_spectral)
  plt.colorbar(cf0)
[ax[j].set_aspect('equal', 'box') for j in range(rank+1)]
# [ax[j+1].set_title(f'Mode {j}, KE = {np.round(KE_mode[j]*100, 3)}%') for j in range(rank)]
plt.tight_layout()
plt.show()